In [87]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [88]:
import pandas as pd, numpy as np, gc

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
orig = pd.read_csv('Exam_Score_Prediction.csv')

print("train_shape:",train.shape)
print("test.shape:",test.shape)
print("orig.shape:",orig.shape)

orig

# 今後のためにリストを作る
target = 'exam_score'
base = [col for col in train.columns if col not in ['id', target]]
categories = train.select_dtypes('object').columns.to_list()
nums = [col for col in base if col not in categories]
print(f'{len(base)} Base Features:{base}')

train_shape: (630000, 13)
test.shape: (270000, 12)
orig.shape: (20000, 13)
11 Base Features:['age', 'gender', 'course', 'study_hours', 'class_attendance', 'internet_access', 'sleep_hours', 'sleep_quality', 'study_method', 'facility_rating', 'exam_difficulty']


In [89]:
ORIG = []

# 外部データの各カラムのユニークごとの平均値というカラムを追加する。
for col in base:
    # 一つの列に対してgroupbyで固有の値をまとめる。それらのtargetをそれぞれ平均する
    mean_map = orig.groupby(col)[target].mean() 
    new_mean_col_name = f"orig_mean_{col}"
    mean_map.name = new_mean_col_name
    
    train = train.merge(mean_map, on=col, how='left') # colをキーにして
    test = test.merge(mean_map, on=col, how='left')
    ORIG.append(new_mean_col_name)
    
# 外部データの各カラムのユニークごとのサイズというカラムを追加する。
    new_count_col_name = f"orig_count_{col}"
    count_map = orig.groupby(col).size().reset_index(name=new_count_col_name)
    
    train = train.merge(count_map, on=col, how='left')
    test = test.merge(count_map, on=col, how='left')
    ORIG.append(new_count_col_name)

print(f'{len(ORIG)} ORIG Features Created.')

22 ORIG Features Created.


In [90]:
# origには存在するが、trainには存在しないカテゴリを全体平均で埋める
for col in ORIG:
    if 'mean' in col:
        train[col] = train[col].fillna(orig[target].mean())
        test[col] = test[col].fillna(orig[target].mean())
    else:
        train[col] = train[col].fillna(0)
        test[col] = test[col].fillna(0)

In [91]:
# reduce_mem_usage はここに定義（そのままでOK）

features = base + ORIG

# まず X, y を作る（これが先）
X = train[features].copy()
y = train[target].copy()

# test側も、モデルに入れる列だけにそろえる（重要）
X_test = test[features].copy()

gc.collect()


44

In [92]:
from sklearn.model_selection import KFold
import numpy as np

# =========================
# Target Encoding (OOFでリーク防止) + 列選別条件つき
# =========================

def select_te_cols(
    df_train, df_test, cols,
    min_unique=3,              # unique <=2 は除外
    max_unique_abs=5000,       # 高カーディナリティ除外
    max_unique_ratio=0.30,     # unique/行数 が大きすぎる列は除外（ID化）
    max_missing=0.60,          # 欠損率が高い列は除外
    rare_thr=5,                # レア判定（出現回数<=5）
    max_rare_points_ratio=0.80,# レアカテゴリが占める割合が大きい列は除外
    max_unseen_ratio=0.20      # testにしかないカテゴリが多い列は除外
):
    n = len(df_train)
    chosen = []
    for col in cols:
        s_tr = df_train[col]
        s_te = df_test[col]

        # 欠損
        if s_tr.isna().mean() > max_missing:
            continue

        # unique
        nunq = s_tr.nunique(dropna=True)
        if nunq < min_unique:
            continue
        if nunq > max_unique_abs:
            continue
        if nunq / n > max_unique_ratio:
            continue

        # レアカテゴリ比率
        vc = s_tr.value_counts(dropna=True)
        rare_points_ratio = (s_tr.map(vc).fillna(0) <= rare_thr).mean()
        if rare_points_ratio > max_rare_points_ratio:
            continue

        # unseen比率（testにあるがtrainにないカテゴリの比率）
        tr_set = set(s_tr.dropna().unique())
        te_set = set(s_te.dropna().unique())
        if len(te_set) > 0:
            unseen_ratio = len(te_set - tr_set) / len(te_set)
            if unseen_ratio > max_unseen_ratio:
                continue

        chosen.append(col)
    return chosen


def add_target_encoding_oof(train_df, test_df, y, te_cols, n_splits=5, seed=42, smoothing=20):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)

    for col in te_cols:
        te_name = f"te_{col}"
        train_te = np.zeros(len(train_df), dtype=np.float64)
        test_te_folds = []

        for tr_idx, va_idx in kf.split(train_df):
            X_tr = train_df.iloc[tr_idx]
            y_tr = y.iloc[tr_idx]
            X_va = train_df.iloc[va_idx]

            prior = y_tr.mean()

            stats = (
                pd.DataFrame({col: X_tr[col].values, "y": y_tr.values})
                .groupby(col)["y"]
                .agg(["mean", "count"])
            )

            smooth_map = (stats["count"] * stats["mean"] + smoothing * prior) / (stats["count"] + smoothing)

            train_te[va_idx] = X_va[col].map(smooth_map).fillna(prior).astype(np.float64).values
            test_te_folds.append(test_df[col].map(smooth_map).fillna(prior).astype(np.float64).values)

        train_df[te_name] = train_te
        test_df[te_name] = np.mean(np.vstack(test_te_folds), axis=0)

    return train_df, test_df


# ---- ここがあなたのコードの差し替え部分 ----

# TE対象を「object列のうち、条件を満たす列」に絞る
TE_COLS_RAW = categories
TE_COLS = select_te_cols(
    train, test, TE_COLS_RAW,
    min_unique=3,
    max_unique_abs=5000,
    max_unique_ratio=0.30,
    max_missing=0.60,
    rare_thr=5,
    max_rare_points_ratio=0.80,
    max_unseen_ratio=0.20
)
print(f"Target Encoding applied to {len(TE_COLS)} features.")
print("TE_COLS:", TE_COLS)

# OOF TE作成
train, test = add_target_encoding_oof(train, test, y, TE_COLS, n_splits=5, seed=42, smoothing=20)

TE_FEATURES = [f"te_{c}" for c in TE_COLS]
features = base + ORIG + TE_FEATURES

X = train[features].copy()
y = train[target].copy()
X_test = test[features].copy()

gc.collect()

print(X.columns)
print(X_test.columns)


Target Encoding applied to 6 features.
TE_COLS: ['gender', 'course', 'sleep_quality', 'study_method', 'facility_rating', 'exam_difficulty']
Index(['age', 'gender', 'course', 'study_hours', 'class_attendance',
       'internet_access', 'sleep_hours', 'sleep_quality', 'study_method',
       'facility_rating', 'exam_difficulty', 'orig_mean_age', 'orig_count_age',
       'orig_mean_gender', 'orig_count_gender', 'orig_mean_course',
       'orig_count_course', 'orig_mean_study_hours', 'orig_count_study_hours',
       'orig_mean_class_attendance', 'orig_count_class_attendance',
       'orig_mean_internet_access', 'orig_count_internet_access',
       'orig_mean_sleep_hours', 'orig_count_sleep_hours',
       'orig_mean_sleep_quality', 'orig_count_sleep_quality',
       'orig_mean_study_method', 'orig_count_study_method',
       'orig_mean_facility_rating', 'orig_count_facility_rating',
       'orig_mean_exam_difficulty', 'orig_count_exam_difficulty', 'te_gender',
       'te_course', 'te_sleep_q

In [93]:
def reduce_mem_usage_safe(df):
    df = df.copy()
    for col in df.columns:
        if df[col].dtype == np.float64:
            df[col] = df[col].astype(np.float32)
        elif df[col].dtype == np.int64:
            df[col] = df[col].astype(np.int32)
    return df

In [94]:
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import numpy as np
import xgboost as xgb
import pandas as pd

# =========================
# 1) one-hot（fold外で1回）
# =========================

# Ridge 用
X_lin = pd.get_dummies(train[features], drop_first=False)
test_lin_X = pd.get_dummies(test[features], drop_first=False)
X_lin, test_lin_X = X_lin.align(test_lin_X, join="left", axis=1, fill_value=0)

# XGB 用
X_xgb = pd.get_dummies(train[features], drop_first=False)
test_xgb = pd.get_dummies(test[features], drop_first=False)
X_xgb, test_xgb = X_xgb.align(test_xgb, join="left", axis=1, fill_value=0)

y = train[target]

# =========================
# 2) CV 設定
# =========================
N_SPLITS = 5
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

oof_lin = np.zeros(len(train))
oof_res = np.zeros(len(train))
test_lin_pred = np.zeros(len(test))
test_res_pred = np.zeros(len(test))

# =========================
# 3) CV ループ
# =========================
for fold, (tr_idx, va_idx) in enumerate(kf.split(X_lin)):

    # ----- Ridge -----
    X_tr_lin, X_va_lin = X_lin.iloc[tr_idx], X_lin.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    lin = Ridge(alpha=1.0, random_state=42)
    lin.fit(X_tr_lin, y_tr)

    y_tr_lin = lin.predict(X_tr_lin)
    y_va_lin = lin.predict(X_va_lin)

    oof_lin[va_idx] = y_va_lin
    test_lin_pred += lin.predict(test_lin_X) / N_SPLITS

    # ----- 残差 -----
    y_tr_res = y_tr - y_tr_lin
    y_va_res = y_va - y_va_lin

    # ----- XGB（残差） -----
    X_tr_xgb, X_va_xgb = X_xgb.iloc[tr_idx], X_xgb.iloc[va_idx]

    xgb_model = xgb.XGBRegressor(
        n_estimators=12000,
        learning_rate=0.02,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42,
        tree_method="hist",
    )

    xgb_model.fit(
        X_tr_xgb, y_tr_res,
        eval_set=[(X_va_xgb, y_va_res)],
        verbose=False
    )

    y_va_res_pred = xgb_model.predict(X_va_xgb)
    oof_res[va_idx] = y_va_res_pred
    test_res_pred += xgb_model.predict(test_xgb) / N_SPLITS

    # ----- Fold RMSE -----
    fold_rmse = np.sqrt(
        mean_squared_error(y_va, y_va_lin + y_va_res_pred)
    )
    print(f"Fold {fold+1} RMSE: {fold_rmse:.5f}")

# =========================
# 4) OOF 評価
# =========================
oof_pred = oof_lin + oof_res
cv_rmse = np.sqrt(mean_squared_error(y, oof_pred))
print("OOF RMSE:", cv_rmse)

# =========================
# 5) Test 予測
# =========================
test_pred = test_lin_pred + test_res_pred

# =========================
# 6) submission 作成
# =========================
# 予測値を範囲に収めたい場合（任意）
test_pred = np.clip(test_pred, y.min(), y.max())

sub = pd.DataFrame({
    "id": test["id"],
    "exam_score": test_pred
})
sub.to_csv("submission.csv", index=False)

print(sub.head())
print("saved: submission.csv")



Fold 1 RMSE: 8.71649
Fold 2 RMSE: 8.71670
Fold 3 RMSE: 8.71520
Fold 4 RMSE: 8.72429
Fold 5 RMSE: 8.72759
OOF RMSE: 8.720055446187217
       id  exam_score
0  630000   69.578297
1  630001   69.974591
2  630002   90.840441
3  630003   55.646008
4  630004   46.875301
saved: submission.csv
